In [1]:
import findspark

findspark.find()
findspark.init()
import pyspark

In [2]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel

In [3]:
sc = pyspark.SparkContext(appName="K-means_MLlib")

In [4]:
# Load and parse the data
data = sc.textFile("data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

In [5]:
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

In [6]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [7]:
WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 0.6928203230275529


In [9]:
# Save and load model
clusters.save(sc, "/home/kira/tmp/Spark/PythonKMeansExample/KMeansModel2")
sameModel = KMeansModel.load(sc, "/home/kira/tmp/Spark/PythonKMeansExample/KMeansModel2")